## Exploring scenarios in which Numba is faster than NumPy or base Python

In [14]:
import numba
from numba import njit
import numpy as np
rng = np.random.default_rng(10031995)

In [59]:
X = np.array([[0., 0], [0, 1], [1, 0]])
y = np.array([.25, 0.3])

In [4]:
@numba.jit
def barycentric(x1, x2, x3, y):
    v0 = y - x1
    v1 = x2 - x1
    v2 = x3 - x1
    d00 = v0.dot(v0)
    d01 = v0.dot(v1)
    d02 = v0.dot(v2)
    d11 = v1.dot(v1)
    d12 = v1.dot(v2)
    d22 = v2.dot(v2)
    det_A = d11 * d22 - d12 * d12
    alpha_1 = (d01 * d22 - d12 * d02) / det_A
    alpha_2 = (d11 * d02 - d01 * d12) / det_A
    return (1 - alpha_1 - alpha_2, alpha_1, alpha_2)

barycentric(*X, y) 

(0.44999999999999996, 0.3, 0.25)

In [5]:
%timeit barycentric(*X, y)

2.71 μs ± 547 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


In [6]:
def barycentric_base(x1, x2, x3, y):
    v0 = y - x1
    v1 = x2 - x1
    v2 = x3 - x1
    d00 = v0.dot(v0)
    d01 = v0.dot(v1)
    d02 = v0.dot(v2)
    d11 = v1.dot(v1)
    d12 = v1.dot(v2)
    d22 = v2.dot(v2)
    det_A = d11 * d22 - d12 * d12
    alpha_1 = (d01 * d22 - d12 * d02) / det_A
    alpha_2 = (d11 * d02 - d01 * d12) / det_A
    return (1 - alpha_1 - alpha_2, alpha_1, alpha_2)

In [9]:
%timeit barycentric_base(*X,y)

8.95 μs ± 542 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Numba is faster than base Python for computing barycentric coordinates.

In [12]:
@njit
def barycentric_njit(x1, x2, x3, y):
    v0 = y - x1
    v1 = x2 - x1
    v2 = x3 - x1
    d00 = v0.dot(v0)
    d01 = v0.dot(v1)
    d02 = v0.dot(v2)
    d11 = v1.dot(v1)
    d12 = v1.dot(v2)
    d22 = v2.dot(v2)
    det_A = d11 * d22 - d12 * d12
    alpha_1 = (d01 * d22 - d12 * d02) / det_A
    alpha_2 = (d11 * d02 - d01 * d12) / det_A
    return (1 - alpha_1 - alpha_2, alpha_1, alpha_2)

barycentric_njit(*X, y) 

(0.44999999999999996, 0.3, 0.25)

In [13]:
%timeit barycentric_njit(*X,y)

2.63 μs ± 408 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Okay, so, when is @njit faster than @numba.jit?

In [62]:
@njit
def barycentric_njit_loops(x1, x2, x3, y, n):
    for _ in range(n):
        v0 = y - x1
        v1 = x2 - x1
        v2 = x3 - x1
        d00 = v0.dot(v0)
        d01 = v0.dot(v1)
        d02 = v0.dot(v2)
        d11 = v1.dot(v1)
        d12 = v1.dot(v2)
        d22 = v2.dot(v2)
        det_A = d11 * d22 - d12 * d12
        alpha_1 = (d01 * d22 - d12 * d02) / det_A
        alpha_2 = (d11 * d02 - d01 * d12) / det_A
        sol = (1 - alpha_1 - alpha_2, alpha_1, alpha_2)
    return None
barycentric_njit_loops(*X, y, 10) 

In [66]:
%timeit barycentric_njit_loops(*X, y, 100) 

28.5 μs ± 5.39 μs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [64]:
def barycentric_base_loops(x1, x2, x3, y, n):
    for _ in range(n):
        v0 = y - x1
        v1 = x2 - x1
        v2 = x3 - x1
        d00 = v0.dot(v0)
        d01 = v0.dot(v1)
        d02 = v0.dot(v2)
        d11 = v1.dot(v1)
        d12 = v1.dot(v2)
        d22 = v2.dot(v2)
        det_A = d11 * d22 - d12 * d12
        alpha_1 = (d01 * d22 - d12 * d02) / det_A
        alpha_2 = (d11 * d02 - d01 * d12) / det_A
        sol = (1 - alpha_1 - alpha_2, alpha_1, alpha_2)
    return None

In [67]:
%timeit barycentric_base_loops(*X, y, 100) 

847 μs ± 148 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


Base Python for loops scale linearly in the loop size. Use of Numba with for loops appears to scale faster than linearly in loop size. At least for this barycentric coordinates example.

In [19]:
def ex1(a, b, out):
    for i in range(a.shape[0]):
        out[i] = a[i] + b[i]
        
a, b = rng.random((2, 3000))
out = np.empty_like(a)

In [20]:
%timeit ex1(a, b, out)

1.03 ms ± 81.7 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [17]:
ex2 = numba.jit(ex1)
ex2(a,b,out)

In [21]:
%timeit ex2(a, b, out)

1.74 μs ± 346 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


Numba is much faster than base Python for for loops.

In [51]:
U = np.random.normal(size=(1000,20))
V = np.random.normal(size=(1000,1))
Us = np.random.normal(size=(1000,1000,20))
Vs = np.random.normal(size=(1000,1000,1))
U.shape, V.shape

((1000, 20), (1000, 1))

In [37]:
@njit
def lstsq_numba(U,V):
    return np.linalg.lstsq(U,V)
out=lstsq_numba(U,V)

In [39]:
%timeit np.linalg.lstsq(U,V,rcond=None)

622 μs ± 32.5 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [40]:
%timeit lstsq_numba(U,V)

711 μs ± 55.4 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)


In [42]:
@njit
def lstsq_numba_loops(Us,Vs):
    N = Us.shape[0]
    for n in range(N):
        U = Us[n]
        V = Vs[n]
        lstsq_numba(U,V)
    return None
lstsq_numba_loops(Us,Vs)

In [52]:
%timeit lstsq_numba_loops(Us, Vs)

685 ms ± 69.8 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [46]:
def lstsq_loops(Us,Vs):
    N = Us.shape[0]
    for n in range(N):
        U = Us[n]
        V = Vs[n]
        np.linalg.lstsq(U,V,rcond=None)
    return None

In [53]:
%timeit lstsq_loops(Us,Vs)

592 ms ± 30 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In the least squares scenarios, it appears that NumPy least squares is already optimized. Previously, with the for loops, Numba is much faster for addition. In the barycentric examples, there are many small operations, and these are faster in Numba than the NumPy options. 